In [6]:
pip install --force-reinstall tabula-py

  Using cached tabula_py-2.3.0-py3-none-any.whl (12.0 MB)
  Using cached distro-1.6.0-py2.py3-none-any.whl (19 kB)
     |████████████████████████████████| 16.9 MB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 11.4 MB 3.5 MB/s eta 0:00:01
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
     |████████████████████████████████| 503 kB 10.6 MB/s eta 0:00:01
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: pytz
    Found existing installation: pytz 2020.1
    Uninstalling pytz-2020.1:
      Successfully uninstalled pytz-2020.1
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
 

In [82]:
import tabula
import os
import pandas as pd
import glob
import numpy as np

In [284]:
tables = tabula.read_pdf("./pdf_files/2019_GRI_Report_DZBANK.pdf",pages="all", stream=True)
print(str(tables))

[    1.3                             Ethics and integrity  11
0   1.4                             Corporate governance  11
1   1.5                           Stakeholder engagement  11
2   1.6                               Reporting practice  14
3   2.0                      GRI 200: Economic standards  17
4   2.1               GRI 201: Economic performance 2016  17
5   2.2          GRI 203: Indirect economic impacts 2016  21
6   2.3                    GRI 205: Anti-corruption 2016  22
7   2.4          GRI 206: Anti-competitive behavior 2016  24
8   3.0                  GRI 300 Environmental standards  25
9   3.1                          GRI 301: Materials 2016  25
10  3.2                             GRI 302: Energy 2016  26
11  3.3                GRI 303: Water and effluents 2018  27
12  3.4                          GRI 305: Emissions 2016  27
13  3.5                GRI 306: Effluents and waste 2016  29
14  3.6           GRI 307: Environmental compliance 2016  30
15  3.7  GRI 308: Suppl

In [285]:
# save them in a folder
folder_name = "modified_tables"
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
# iterate over extracted tables and export as excel individually
for i, table in enumerate(tables, start=1):
    table = pd.DataFrame(table)
    new_header = table.iloc[0] #grab the first row for the header
    table = table[1:] #take the data less the header row
    table.columns = new_header #set the header row as the df header
    #table = table.to_string(index=False)
    table.rename(columns={ table.columns[0]: "Year" }, inplace = True)
    table = table.transpose()
    print(table)
    table.to_excel(os.path.join(folder_name, f"table_{i}.xlsx"), header = False)

                                          1                   2   \
0                                                                  
Year                                     1.5                 1.6   
Corporate governance  Stakeholder engagement  Reporting practice   
11                                        11                  14   

                                               3   \
0                                                   
Year                                          2.0   
Corporate governance  GRI 200: Economic standards   
11                                             17   

                                                      4   \
0                                                          
Year                                                 2.1   
Corporate governance  GRI 201: Economic performance 2016   
11                                                    17   

                                                           5   \
0                          

                         1                                    2  \
0                                                                 
Year    Total participants  Total for company sports activities   
2019.0               784.0                                554.0   
2018.0               991.0                                683.0   
2017                 1,061                                  740   

                                                   3  \
0                                                      
Year    Total for sickness/injury prevention courses   
2019.0                                         230.0   
2018.0                                         308.0   
2017                                             321   

                                           4  
0                                             
Year    Average health rate for the year (%)  
2019.0                                  95.3  
2018.0                                  95.3  
2017                     

In [286]:
folder_name = "result"
dfs = pd.DataFrame()
for file_name in glob.glob("modified_tables/*.xlsx"):
    df = pd.read_excel(file_name)
    df = pd.DataFrame(df)
    if dfs.empty:
        dfs = df.copy()
    else:
        dfs = df.merge(dfs,left_index=True, right_index=True,
                 how='outer', suffixes=('', '_y'))
        dfs.drop(dfs.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        dfs = dfs.dropna(how='all', axis=1)
dfs.to_excel(os.path.join(folder_name,"results.xlsx"), header = True, index = False)